In [ ]:
#Installations
#Translation, Summarization and Text Generation
!pip install evaluate
!pip install transformers sentencepiece datasets
!pip install rouge_score
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 8.6 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 86.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.0 MB/s eta 0:00:00


In [ ]:
#Translation, Summarization and Text Generation
from datasets import load_dataset
from datasets import concatenate_datasets, DatasetDict
from google.colab import drive
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import nltk
import evaluate
import os
from torch import optim
from torch.nn import functional as F
from transformers import AdamW, AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import get_linear_schedule_with_warmup
from transformers import pipeline
from tqdm import tqdm_notebook

sns.set()

#Sumarization 

In [ ]:
from datasets import load_dataset

ds = load_dataset("csebuetnlp/xlsum", name="spanish")
ds

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset xlsum downloaded and prepared to /root/.cache/huggingface/datasets/csebuetnlp___xlsum/spanish/2.0.0/518ab0af76048660bcc2240ca6e8692a977c80e384ffb18fdddebaca6daebdce. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 38110
    })
    test: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 4763
    })
    validation: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 4763
    })
})

In [ ]:
ds["train"][0]

{'id': '140930_ultnot_siria_onu_comida_ch',
 'url': 'https://www.bbc.com/mundo/ultimas_noticias/2014/09/140930_ultnot_siria_onu_comida_ch',
 'title': 'ONU dice que peligran raciones de comida para Siria',
 'summary': 'La directora de ayuda humanitaria de Naciones Unidas, Valerie Amos, advirtió que de no invertir más dinero, el Programa Mundial de Alimentos tendrá que detener sus operaciones en Siria en dos meses.',
 'text': 'De no recibir más dinero, las raciones de la ONU en Siria se terminarán en dos meses. En un informe al Consejo de Seguridad de la ONU, Amos dijo que las raciones del PMA destinadas a los 4.000.000 de sirios ya han sido recortadas para poder llegar a la mayor cantidad de personas posible. Amos también hizo un llamado a juntar suministros para proteger a los sirios del frío, en miras al próximo invierno.'}

In [ ]:
t5_tokenizer = AutoTokenizer.from_pretrained("google/mt5-base")

Downloading:   0%|          | 0.00/376 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/702 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
def tokenize_sample_data(data):
  # Max token size is 14536 and 215 for inputs and labels, respectively.
  # Here I restrict these token size.
  input_feature = t5_tokenizer(data["text"], truncation=True, max_length=1024)
  label = t5_tokenizer(data["summary"], truncation=True, max_length=128)
  return {
    "input_ids": input_feature["input_ids"],
    "attention_mask": input_feature["attention_mask"],
    "labels": label["input_ids"],
  }

tokenized_ds = ds.map(
  tokenize_sample_data,
  remove_columns=["id", "url", "title", "summary", "text"],
  batched=True,
  batch_size=128)

tokenized_ds

  0%|          | 0/298 [00:00<?, ?ba/s]

  0%|          | 0/38 [00:00<?, ?ba/s]

  0%|          | 0/38 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 38110
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 4763
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 4763
    })
})

In [ ]:
import torch
from transformers import AutoConfig, AutoModelForSeq2SeqLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# see https://huggingface.co/docs/transformers/main_classes/configuration
mt5_config = AutoConfig.from_pretrained(
  "google/mt5-base",
  max_length=128,
  length_penalty=0.6,
  no_repeat_ngram_size=2,
  num_beams=15,
)
model = (AutoModelForSeq2SeqLM
         .from_pretrained("google/mt5-base", config=mt5_config)
         .to(device))

Downloading:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
  t5_tokenizer,
  model=model,
  return_tensors="pt")

In [ ]:
import evaluate
import numpy as np
from nltk.tokenize import RegexpTokenizer

rouge_metric = evaluate.load("rouge")

# define function for custom tokenization
def tokenize_sentence(arg):
  encoded_arg = t5_tokenizer(arg)
  return t5_tokenizer.convert_ids_to_tokens(encoded_arg.input_ids)

# define function to get ROUGE scores with custom tokenization
def metrics_func(eval_arg):
  preds, labels = eval_arg
  # Replace -100
  labels = np.where(labels != -100, labels, t5_tokenizer.pad_token_id)
  # Convert id tokens to text
  text_preds = t5_tokenizer.batch_decode(preds, skip_special_tokens=True)
  text_labels = t5_tokenizer.batch_decode(labels, skip_special_tokens=True)
  # Insert a line break (\n) in each sentence for ROUGE scoring
  # (Note : Please change this code, when you perform on other languages except for Japanese)
  text_preds = [(p if p.endswith(("!", "！", "?", "？", ".")) else p + ".") for p in text_preds]
  text_labels = [(l if l.endswith(("!", "！", "?", "？", ".")) else l + ".") for l in text_labels]
  sent_tokenizer_jp = RegexpTokenizer(u'[^!！?？.]*[!！?？.]')
  text_preds = ["\n".join(np.char.strip(sent_tokenizer_jp.tokenize(p))) for p in text_preds]
  text_labels = ["\n".join(np.char.strip(sent_tokenizer_jp.tokenize(l))) for l in text_labels]
  # compute ROUGE score with custom tokenization
  return rouge_metric.compute(
    predictions=text_preds,
    references=text_labels,
    tokenizer=tokenize_sentence
  )

In [ ]:
from torch.utils.data import DataLoader

sample_dataloader = DataLoader(
  tokenized_ds["test"].with_format("torch"),
  collate_fn=data_collator,
  batch_size=5)
for batch in sample_dataloader:
  with torch.no_grad():
    preds = model.generate(
      batch["input_ids"].to(device),
      num_beams=15,
      num_return_sequences=1,
      no_repeat_ngram_size=1,
      remove_invalid_values=True,
      max_length=128,
    )
  labels = batch["labels"]
  break

metrics_func([preds, labels])

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'rouge1': 0.23279235706821916,
 'rouge2': 0.0705992509363296,
 'rougeL': 0.2055127401334298,
 'rougeLsum': 0.21081120977972553}

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
  output_dir = "mt5-summarize-es",
  log_level = "error",
  num_train_epochs = 10,
  learning_rate = 5e-4,
  lr_scheduler_type = "linear",
  warmup_steps = 90,
  optim = "adafactor",
  weight_decay = 0.01,
  per_device_train_batch_size = 2,
  per_device_eval_batch_size = 1,
  gradient_accumulation_steps = 16,
  evaluation_strategy = "steps",
  eval_steps = 100,
  predict_with_generate=True,
  generation_max_length = 128,
  save_steps = 500,
  logging_steps = 10,
  push_to_hub = False
)

In [ ]:
from transformers import Seq2SeqTrainer
trainer = Seq2SeqTrainer(
  model = model,
  args = training_args,
  data_collator = data_collator,
  compute_metrics = metrics_func,
  train_dataset = tokenized_ds["train"],
  eval_dataset = tokenized_ds["validation"].select(range(20)),
  tokenizer = t5_tokenizer,
)

trainer.train()


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
100,3.223100,2.616407,0.267486,0.080655,0.210721,0.209704
200,3.040800,2.602019,0.262536,0.082622,0.207583,0.209308
300,2.876900,2.550814,0.327884,0.139808,0.255954,0.255907
400,2.899500,2.555597,0.304747,0.133548,0.242984,0.246322
500,2.793300,2.519356,0.280553,0.122233,0.232961,0.234603
600,2.807300,2.514755,0.328133,0.130181,0.258039,0.257951
700,2.753700,2.545967,0.300933,0.131636,0.254587,0.254069
800,2.735600,2.473975,0.322099,0.136673,0.271996,0.271835
900,2.728300,2.465037,0.319721,0.140782,0.249201,0.250154
1000,2.596800,2.441478,0.278671,0.116072,0.230026,0.230134


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
100,3.223100,2.616407,0.267486,0.080655,0.210721,0.209704
200,3.040800,2.602019,0.262536,0.082622,0.207583,0.209308
300,2.876900,2.550814,0.327884,0.139808,0.255954,0.255907
400,2.899500,2.555597,0.304747,0.133548,0.242984,0.246322
500,2.793300,2.519356,0.280553,0.122233,0.232961,0.234603
600,2.807300,2.514755,0.328133,0.130181,0.258039,0.257951
700,2.753700,2.545967,0.300933,0.131636,0.254587,0.254069
800,2.735600,2.473975,0.322099,0.136673,0.271996,0.271835
900,2.728300,2.465037,0.319721,0.140782,0.249201,0.250154
1000,2.596800,2.441478,0.278671,0.116072,0.230026,0.230134


TrainOutput(global_step=11900, training_loss=2.0068873308486297, metrics={'train_runtime': 41377.6551, 'train_samples_per_second': 9.21, 'train_steps_per_second': 0.288, 'total_flos': 8.625871923998884e+17, 'train_loss': 2.0068873308486297, 'epoch': 10.0})

In [ ]:
import os
from transformers import AutoModelForSeq2SeqLM

# save fine-tuned model in local
os.makedirs("./trained_for_summarization_es", exist_ok=True)
if hasattr(trainer.model, "module"):
  trainer.model.module.save_pretrained("./trained_for_summarization_es")
else:
  trainer.model.save_pretrained("./trained_for_summarization_es")

# load local model
model = (AutoModelForSeq2SeqLM
         .from_pretrained("./trained_for_summarization_es")
         .to(device))

In [ ]:
from torch.utils.data import DataLoader

# Predict with test data (first 5 rows)
sample_dataloader = DataLoader(
  tokenized_ds["test"].with_format("torch"),
  collate_fn=data_collator,
  batch_size=5)
for batch in sample_dataloader:
  with torch.no_grad():
    preds = model.generate(
      batch["input_ids"].to(device),
      num_beams=15,
      num_return_sequences=1,
      no_repeat_ngram_size=1,
      remove_invalid_values=True,
      max_length=128,
    )
  labels = batch["labels"]
  break

# Replace -100 (see above)
labels = np.where(labels != -100, labels, t5_tokenizer.pad_token_id)

# Convert id tokens to text
text_preds = t5_tokenizer.batch_decode(preds, skip_special_tokens=True)
text_labels = t5_tokenizer.batch_decode(labels, skip_special_tokens=True)

# Show result
print("***** Input's Text *****")
print(ds["test"]["text"][2])
print("***** Summary Text (True Value) *****")
print(text_labels[2])
print("***** Summary Text (Generated Text) *****")
print(text_preds[2])

***** Input's Text *****
Los algoritmos son como manos invisibles que toman decisiones por nosotros. El diccionario de la Real Academia Española lo define como "un conjunto ordenado y finito de operaciones que permite hallar la solución de un problema". Pero en la sociedad contemporánea, los algoritmos han adquirido un rol cada vez más central, haciendo que algunos los definan como una suerte de mano invisible que toma decisiones por nosotros. "Todo lo que la gente hace y ve en la web es producto de algoritmos", explican en un informe expertos del centro independiente de investigación Pew Research Center, con sede en Washington, Estados Unidos. En otras palabras, son el mecanismo oculto responsable de lo que vemos en el muro Facebook, las sugerencias que nos muestra Netflix o los resultados que aparecen cuando introducimos una palabra en el buscador de Google. Final de Quizás también te interese Los algoritmos están detrás de mucho de lo que vemos, o no, en la red. Y, a veces, pueden i